# Proyecto Demo: Análisis de Datos y Clustering

Este notebook tiene como objetivo realizar un análisis de datos, aplicar técnicas de clustering y exportar los resultados a una base de datos de PostreSQL utilizando Supabase.

In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    !pip install supabase

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import json
from supabase import create_client

In [ ]:
# Leer datos desde el archivo CSV
url = "https://raw.githubusercontent.com/jorgermzg15/data-files/refs/heads/main/Mall_Customers.csv"
data = pd.read_csv(url)
data.head()

In [ ]:
# Preprocesamiento de datos
# Eliminar valores nulos y convertir datos categóricos si es necesario
data = data.dropna()
data['Genre'] = data['Genre'].map(
    {
        'Male': 0, 
        'Female': 1
    }
)
data.head()

In [ ]:
# Aplicar clustering con KMeans
X = data[['Age', 'Annual Income (k$)', 'Spending Score (1-100)']]

kmeans = KMeans(n_clusters=5, random_state=42)
labels = kmeans.fit_predict(X)

clusters = pd.DataFrame(
    {
        'CustomerID': data.CustomerID,
        'Cluster': labels
    }
)
clusters.head()

In [ ]:
if IN_COLAB:
  from google.colab import userdata
  supabase = create_client(
    userdata.get('SUPABASE_URL'), 
    userdata.get('SUPABASE_KEY')
  )

else:
  # Si credentials.json no existe, crealo y agrega SUPABASE_URL y SUPABASE_KEY
  with open("credentials.json", "r") as f:
    config = json.load(f)

  url = config["SUPABASE_URL"]
  key = config["SUPABASE_KEY"]

  supabase = create_client(url, key)

In [ ]:
# Sobrescribir datos: eliminar todo antes de insertar (respetando FK)
# Primero borrar cluster_labels (hija), luego observations (padre)
supabase.table("cluster_labels").delete().neq("customer_id", None).execute()
supabase.table("observations").delete().neq("customer_id", None).execute()

# Insertar primero observations (padre)
supabase.table("observations").insert(
    data.rename(columns={
        "CustomerID": "customer_id",
        "Genre": "genre",
        "Age": "age",
        "Annual Income (k$)": "annual_income_k",
        "Spending Score (1-100)": "spending_score"
    }).to_dict(orient="records")
).execute()

# Luego insertar cluster_labels (hija)
supabase.table("cluster_labels").insert(
    clusters.rename(columns={
        "CustomerID": "customer_id",
        "Cluster": "cluster"
    }).to_dict(orient="records")
).execute()